Import necessary modules

In [20]:
from mistralai import Mistral
from helpers.config import Config
import functools
import json
import requests

Initiate Mistral AI Client

In [21]:
MISTRAL_API_KEY = Config.MISTRAL_API_KEY
client = Mistral(api_key=MISTRAL_API_KEY)

Define Function Schema

In [22]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get Today's Weather in Specified Location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location for the weather forecast.",
                    }
                },
                "required": ["location"],
            },
        },
    },
]

Define Function

In [23]:
def get_weather(location: str) -> str:
    api_key = "777c42660156447db5842748240110"
    result = requests.get(
        f"https://api.weatherapi.com/v1/current.json?key={api_key}&q={location}"
    )

    return result.json()

Define Name to Function Mapping

In [24]:
names_to_functions = {"get_weather": functools.partial(get_weather)}

Prepare Prompt Template

In [25]:
messages = []
messages.append(
    {
        "role": "system",
        "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.",
    }
)
messages.append({"role": "user", "content": "How is the weather in Yangon?"})

Get Tool Response

In [ ]:
response = client.chat.complete(
    model="open-mixtral-8x22b", messages=messages, tools=tools
)
tool_call_response = response.choices[0].message
messages.append(tool_call_response)
tool_call_response

Invoke Function Calls and Append Tool Response To Messages

In [ ]:
tool_call = response.choices[0].message.tool_calls[0]
function_name = tool_call.function.name
function_params = json.loads(tool_call.function.arguments)
function_result = names_to_functions[function_name](function_params)

messages.append(
    {
        "role": "tool",
        "name": function_name,
        "content": json.dumps(function_result),
        "tool_call_id": tool_call.id,
    }
)
messages

Inovke LLM using Tool Response

In [ ]:
response = client.chat.complete(model="open-mixtral-8x22b", messages=messages)
response.choices[0].message.content